In [1]:
!pip install pythreejs
!pip install notebook
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user


^C



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
[I 14:21:20 InstallContribNbextensionsApp] jupyter contrib nbextension install --user
[I 14:21:20 InstallContribNbextensionsApp] Installing jupyter_contrib_nbextensions nbextension files to jupyter data directory
[I 14:21:20 InstallContribNbextensionsApp] Installing c:\Users\bluec\AppData\Local\Programs\Python\Python312\Lib\site-packages\jupyter_contrib_nbextensions\nbextensions\addbefore -> addbefore
[I 14:21:20 InstallContribNbextensionsApp] Up to date: C:\Users\bluec\AppData\Roaming\jupyter\nbextensions\addbefore\addbefore.yaml
[I 14:21:20 InstallContribNbextensionsApp] Up to date: C:\Users\bluec\AppData\Roaming\jupyter\nbextensions\addbefore\icon.png
[I 14:21:20 InstallContribNbextensionsApp] Up to date: C:\Users\bluec\AppData\Roaming\jupyter\nbextensions\addbefore\main.js
[I 14:21:20 InstallContribNbextensionsApp] Up to date: C:\Users\bluec\AppData\Roaming\jup

In [1]:
from pythreejs import *
from IPython.display import display

# Create a 3D sphere
sphere = Mesh(
    geometry=SphereGeometry(radius=1),
    material=MeshStandardMaterial(color='red')
)

# Rendering
width, height = 1600, 900

light = PointLight(position=[10, 10, 10], intensity=1.5)
scene = Scene(children=[sphere, light, AmbientLight(intensity=0.5)])


camera = PerspectiveCamera(position=[3, 3, 3], fov=50, aspect=width/height)
controller = OrbitControls(controlling=camera)


renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
display(renderer)


Renderer(camera=PerspectiveCamera(aspect=1.7777777777777777, position=(3.0, 3.0, 3.0), projectionMatrix=(1.0, …

# Patricle Simulation

In [3]:
from pythreejs import * 
from IPython.display import display

import ipywidgets as widgets
import numpy as np


def generate_subdivided_circle(radius=1.0, segments=16):
    """Generate vertices and triangular faces for a subdivided 2D circle."""
    vertices = []
    faces = []
    
    # Add the center vertex
    vertices.append([0.0, 0.0, 0.0])  # Center at (0, 0)
    
    # Generate boundary vertices
    for i in range(segments):
        theta = 2 * np.pi * i / segments
        x = radius * np.cos(theta)
        y = radius * np.sin(theta)
        vertices.append([x, y, 0.0])
    
    # Create faces (triangles)
    for i in range(segments):
        p1 = 0  # Center vertex
        p2 = i + 1
        p3 = (i + 1) % segments + 1  # Wrap around to the first boundary vertex
        faces.append([p1, p2, p3])
    
    return np.array(vertices, dtype=np.float32), np.array(faces, dtype=np.uint32)

# Generate sphere vertices and faces
radius, segments = 1.0, 64
vertices, faces = generate_subdivided_circle(radius=radius, segments=segments)
vertices[:, 1] += 3
# Create BufferGeometry using the vertices and faces
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(array=vertices, normalized=False),
    },
    index=BufferAttribute(array=faces.ravel(), normalized=False)
)

# Define material and mesh
material = MeshStandardMaterial(color='black', wireframe=True)
mesh = Mesh(geometry=geometry, material=material)



# Draw Ground
ground_vertices = np.array([
    [-10.0, 0.0, 0.0],  # Start point
    [10.0, 0.0, 0.0]    # End point
], dtype=np.float32)
ground_geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(ground_vertices, normalized=False)
    }
)
ground_material = LineBasicMaterial(color='red', linewidth=2)
line = Line(geometry=ground_geometry, material=ground_material)



# Rendering setup
width, height = 800, 600  # Smaller dimensions for 2D
aspect = width / height
scene = Scene(children=[mesh, line, AmbientLight(intensity=0.8)])
camera = OrthographicCamera(left=-5*aspect, right=5*aspect, top=5, bottom=-5, near=0.001, far=100, position=[0, 0, 10])
controller = OrbitControls(controlling=camera, enableRotate=False)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
renderer.background = '#FFFFFF'  # Optional: Set a white background for better 2D visibility


button = widgets.Button(desciption="RESET")  
output = widgets.Output()


def on_button_click(b):
    global vertices, faces, geometry
    with output:
        # Regenerate the circle
        vertices, faces = generate_subdivided_circle(radius=np.random.uniform(0.5, 2.0), segments=segments)
        vertices[:, 1] += 3  # Adjust position
        
        # Update geometry
        geometry.attributes['position'].array = vertices
        geometry.index.array = faces.ravel()
        geometry.attributes['position'].needsUpdate = True
        geometry.index.needsUpdate = True

button.on_click(on_button_click)

# Display the renderer and button
display(widgets.VBox([renderer, button, output]))

In [ ]:
import numpy as np
import time

# Constraints (for example, distance constraints between vertices)
def generate_constraints(vertices, faces):
    # Step 1: Extract unique edges from faces
    edges = set()
    for face in faces:
        # Add each edge of the triangle, sorted to avoid duplicates
        edges.add(tuple(sorted((face[0], face[1]))))
        edges.add(tuple(sorted((face[1], face[2]))))
        edges.add(tuple(sorted((face[2], face[0]))))
    
    # Step 2: Generate constraints
    constraints = []
    for edge in edges:
        v1, v2 = edge
        rest_length = np.linalg.norm(vertices[v1] - vertices[v2])
        constraints.append((v1, v2, rest_length))
    
    return constraints


def apply_external_force(velocities, gravity, time_step):
    v = velocities + inv_mass* gravity * time_step
    p = positions + v * time_step
    return p

def solve_constraints(positions, constraints, inv_mass, num_iterations):
    for _ in range(num_iterations):
        for (i, j, rest_length) in constraints:
            p1, p2 = positions[i], positions[j]
            delta = p2 - p1
            delta_length = np.linalg.norm(delta)
            if delta_length > 0:
                correction = (delta_length - rest_length) * delta / delta_length
                positions[i] += correction * inv_mass / 2
                positions[j] -= correction * inv_mass / 2
    return positions


# State
positions = vertices.copy()
velocities = np.zeros_like(positions)

# Physics parameters
mass = 1.0
inv_mass = 1.0 / mass
constraints = generate_constraints(vertices, faces)

# Simulation loop
# Simulation parameters
time_step = 1/30
num_iterations = 10
gravity = np.array([0, -9.81, 0])
for step in range(10000):  # Run for 100 steps
    positions = apply_external_force(velocities, gravity, time_step)
    positions = solve_constraints(positions, constraints, inv_mass, num_iterations)
    
    collide = np.where(positions[:, 1] < 0)
    positions[collide, 1] = 0
    # Update velocities based on new positions
    velocities = (positions - vertices) / time_step
    vertices = positions.copy()
    
    # Dynamic friction
    velocities[collide, 0] = velocities[collide, 0] * np.maximum(1 - 0.2*time_step*abs(velocities[collide, 0]), 0)
    # Static friction
    condition = np.abs(velocities[collide, 0]) < 0.9 * np.abs(velocities[collide, 1] + 1e-6)
    velocities[collide, 0] = np.where(condition, 0, velocities[collide, 0])
    
    # Render the updated positions (this part depends on your rendering setup)
    # For example, you can update the BufferGeometry with new positions
    geometry.attributes['position'].array = positions
    geometry.attributes['position'].needs_update = True
    time.sleep(0.01)

c:\Users\bluec\AppData\Local\Programs\Python\Python312\Lib\site-packages\pythreejs\traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


KeyboardInterrupt: 

# Rigid Body Simulation

In [ ]:
class RigidBody:
    def __init__(self, vertices, faces, mass=1.0):
        self.vertices = vertices
        self.faces = faces
        
        # Mass properties
        self.mass = mass
        self.invMass = 1.0 / mass if mass != 0 else 0
        
        self.inertia = np.eye(3)
        self.invInertia = np.linalg.inv(self.inertia)
        
        # Linear state
        self.currPos = vertices.copy()
        self.prevPos = vertices.copy()
        self.currVel = np.zeros_like(vertices)
        self.prevVel = np.zeros_like(vertices)
        
        # Angular state
        self.currRot = np.array([1.0, 0.0, 0.0, 0.0])  # Quaternion [w, x, y, z]
        self.prevRot = np.array([1.0, 0.0, 0.0, 0.0])
        self.currAngVel = np.zeros(3)
        self.prevAngVel = np.zeros(3)
        
        


In [2]:
def quaternion_multiply(quaternion1, quaternion0):
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)

def BodyPairCorrection(body1, body2, correction, r1, r2, compliance, h, constraints_type):  
    
    if constraints_type == 'positional':
        # Positional Constraints    
        # correction is a linear vector
        c = np.linalg.norm(correction)
        if c < 1e-5:
            return 
        normal = correction / c    
        
        w1 = body1.invMass + np.cross(r1, normal)[None] @ body1.invInertia @ np.cross(r1, normal) # (2)
        w2 = body2.invMass + np.cross(r2, normal)[None] @ body2.invInertia @ np.cross(r2, normal) # (3)
        
        init_lambda = 0
        alpha_tilde = compliance / h**2
        dlambda = -c - init_lambda*alpha_tilde  / (w1 + w2 + alpha_tilde) # (4)
        ## Positional Impulse
        p = dlambda * normal 
        
        ### (6), (7)
        body1.currPos += body1.invMass * p
        body2.currPos -= body2.invMass * p
        
        angle_correction1 = body1.invInertia @ np.cross(r1, p)
        angle_correction2 = body2.invInertia @ np.cross(r2, p)
        ### (8)
        body1.currRot += 0.5 * quaternion_multiply(np.array([0, angle_correction1[0], angle_correction1[1], angle_correction1[2]]), body1.currRot)
        body1.currRot /= np.linalg.norm(body1.currRot)
        ### (9)
        body2.currRot -= 0.5 * quaternion_multiply(np.array([0, angle_correction2[0], angle_correction2[1], angle_correction2[2]]), body2.currRot)
        body2.currRot /= np.linalg.norm(body2.currRot)
    
    elif constraints_type == 'rotational':
        # Rotational Constraints
        # correction is an angular velocity vector
        # r1, r2 are None for this case
        angle = np.linalg.norm(correction)
        if angle < 1e-5:
            return
        axis = correction / angle
        
        w1 = axis[None] @ body1.invInertia @ axis # (11) 
        w2 = axis[None] @ body2.invInertia @ axis # (12)
        
        init_lambda = 0
        alpha_tilde = compliance / h**2
        dlambda = -angle - init_lambda*alpha_tilde  / (w1 + w2 + alpha_tilde) # (13)
        ## Rotational Impulse
        p = dlambda * axis
        
        angle_correction1 = body1.invInertia @ p
        angle_correction2 = body2.invInertia @ p
        ### (15)
        body1.currRot += 0.5 * quaternion_multiply(np.array([0, angle_correction1[0], angle_correction1[1], angle_correction1[2]]), body1.currRot)
        body1.currRot /= np.linalg.norm(body1.currRot)
        ### (16)
        body2.currRot -= 0.5 * quaternion_multiply(np.array([0, angle_correction2[0], angle_correction2[1], angle_correction2[2]]), body2.currRot)
        body2.currRot /= np.linalg.norm(body2.currRot)
    

def collect_collisions():
    pass

def integrate(body, gravity, h):
    # Update Linear state
    body.prevPos = body.currPos
    body.currVel += h * gravity * body.invMass
    body.currPos += h * body.currVel
    
    # Update Angular state
    body.prevRot = body.currRot
    body.currAngVel += h * body.invInertia @ (body.torque - np.cross(body.currAngVel, body.inertia @ body.currAngVel))
    body.currRot += h * 0.5 * quaternion_multiply(np.array([0, body.currAngVel[0], body.currAngVel[1], body.currAngVel[2]]), body.currRot)
    body.currRot /= np.linalg.norm(body.currRot)
    
def solve_constraints(constraints):
    for constraint in constraints:
        constraint.solve()
        
def update_velocities(body):
    body.currVel = (body.currPos - body.prevPos)/h
    body.currAngVel = (body.currRot - body.prevRot)/h

def solve_velocities():
    pass    



# World Setup
gravity = np.array([0, -9.81, 0])
bodies = []

constraints = []
eps = 1e-6

# Simulation parameters
time_step = 1/30
substeps = 10
h = time_step/substeps
while True:
    collisons = collect_collisions(bodies)
    
    for _ in range(substeps):
        for body in bodies:
            integrate(body, gravity, h) # update position and orientation with external forces and torques
            
        ## Solve Positions
        for constraint in constraints:
            solve_constraints()
        # solve_contacts()
        
        ## Updata Velocities with solved positions
        for body in bodies:
            update_velocities()
            
        ## Solve Velocities  - Damping and Friction
        solve_velocities()
    
    render()
    time.sleep(0.01)

NameError: name 'ef' is not defined